In [1]:
import numpy as np
import pandas as pd
import h5py

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [3]:
file = 'C:/Users/CCheny/Documents/UC San Diego - Junction/Bioinformatics/MS2-Autoencoder/chemical_split_data.hdf5'
f = h5py.File(file, 'r')
dataset_low = f['low_peaks']
dataset_high = f['high_peaks']
print(dataset_low.shape)
print(dataset_high.shape)

(110881, 2000)
(110881, 2000)


In [4]:
def generator(X_data, y_data, batch_size):
    print('generator initiated')
    steps_per_epoch = X_data.shape[0]
    numbers_of_epochs = steps_per_epoch // batch_size
    i = 0
    
    while True:
        X_batch = dataset_low[i*batch_size:(i+1)*batch_size]
        y_batch = dataset_high[i*batch_size:(i+1)*batch_size]
        i += 1
        yield X_batch, y_batch
        print('\ngenerator yielded a batch %s' %i)
        if i >= number_of_batches:
                i = 0

In [6]:
input_size = 2000

input_scan = Input(shape=(input_size, 1))
print(input_scan.shape)
hidden_1 = Conv1D(1000, (5, ), activation='relu', padding='same')(input_scan)
print(hidden_1.shape)
hidden_2 = MaxPooling1D()(hidden_1)
print(hidden_2.shape)
hidden_3 = Conv1D(500, (5, ), activation='relu', padding='same')(hidden_2)
print(hidden_3.shape)
hidden_4 = MaxPooling1D()(hidden_3)
print(hidden_4.shape)
hidden_5 = Conv1D(100, (5, ), activation='relu', padding='same')(hidden_4)
print(hidden_5.shape)
encoded = MaxPooling1D()(hidden_5)
print(encoded.shape)

hidden_6 = Conv1D(100, (5, ), activation='relu', padding='same')(encoded)
print(hidden_6.shape)
hidden_7 = UpSampling1D()(hidden_6)
print(hidden_7.shape)
hidden_8 = Conv1D(500, (5, ), activation='relu', padding='same')(hidden_7)
print(hidden_8.shape)
hidden_9 = UpSampling1D()(hidden_8)
print(hidden_9.shape)
hidden_10 = Conv1D(1000, (5, ), activation='relu', padding='same')(hidden_9)
print(hidden_10.shape)
hidden_11 = UpSampling1D()(hidden_10)
print(hidden_11.shape)
decoded = Conv1D(2000, (5, ), activation='relu', padding='same')(hidden_11)
print(decoded.shape)

autoencoder = Model(input_scan, decoded)
autoencoder.compile(optimizer='adadelta', loss='cosine_proximity')

(?, 2000, 1)
(?, 2000, 1000)

(?, 1000, 1000)
(?, 1000, 500)
(?, 500, 500)
(?, 500, 100)
(?, 250, 100)
(?, 250, 100)
(?, 500, 100)
(?, 500, 500)
(?, 1000, 500)
(?, 1000, 1000)
(?, 2000, 1000)
(?, 2000, 2000)


In [7]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2000, 1)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2000, 1000)        6000      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1000, 1000)        0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 500)         2500500   
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 500, 500)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 100)          250100    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 250, 100)          0   

In [8]:
batch_size = 100
gen = generator=generator(dataset_low, dataset_high, batch_size)
autoencoder.fit_generator(gen, 
                         max_queue_size=10,
                         steps_per_epoch=dataset_low.shape[0] // batch_size,
                         epochs=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
generator initiated

generator yielded a batch 1


ValueError: Error when checking input: expected input_2 to have 3 dimensions, but got array with shape (100, 2000)